### 資料讀入

In [110]:
# 套件導入
import time
import os
import pandas as pd
import numpy as np
from gurobipy import *

# 結果資料夾建立
# 如果不存在則創建目錄輸出
if not os.path.isdir('output'): 
    os.mkdir('output')

# 題庫讀取
df = pd.read_csv("./data.txt", header=None)
# print(df)

# print(file)
# s = pd.Series(file)

# 轉換資料
# str => str_list
data = df.values.tolist()
data_0 = data[0][0].split(" ")
data_1 = data[1][0].split("  ")
data_2 = data[2][0].split(" ")
data_3 = data[3][0].split(" ")
# print(data_0)
# print(data_1)
# print(data_2)
# print(data_3)

# 用for 跑暫時無解 data.txt => str_of_list => num_of_list
# for i in file:
    # print(i)
    # print(type(i))

    # list_str = i.split()
    # list(map(int,list_str[0]))
    # print(list_str)

# str_list => num_list
# 第一列
data_0_list_num = list(map(int,data_0))
print(f"第0列: {data_0_list_num},資料型態: {type(data_0_list_num)}")
# 第二列
data_1_list_num = list(map(int,data_1))
print(f"第1列: {data_1_list_num},資料型態: {type(data_1_list_num)}")
# 第三列
data_2_list_num = list(map(int,data_2))
print(f"第2列: {data_2_list_num},資料型態: {type(data_2_list_num)}")
# 第四列
data_3_list_num = list(map(int,data_3))
print(f"第3列: {data_3_list_num},資料型態: {type(data_3_list_num)}")
# print(f"第一部分:{list_num[0]}")
# print(f"第一部分:{list_num[1]}")
# print(f"第一部分:{list_num[2]}")

第0列: [15],資料型態: <class 'list'>
第1列: [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120],資料型態: <class 'list'>
第2列: [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240],資料型態: <class 'list'>
第3列: [750],資料型態: <class 'list'>


### 宣告變數
變數<br>
sample:數量<br>
c:價值<br>
A:重量<br>
b:背包容積<br>


In [111]:
# 定義
# 物品數量
sample = data_0_list_num[0]
print(sample)
# 物品價值
c = np.array(data_2_list_num)
print(c[3])
print(type(int(c[3])))
# 物品重量
A = np.array(data_1_list_num)
print(A[3])
# 背包容積
b = data_3_list_num[0]
print(b)
# 物品數量
print(c.shape[0], A.shape[0])


15
150
<class 'int'>
80
750
15 15


### 跑Gurobi_Model
變數<br>
model:選擇的模型<br>
x: 目前未知<br>
obj: 計算的多項式<br>



In [112]:
# 模型設定
model = Model("Knapsack")
# variables
x = model.addVars(c.shape[0], lb = 0, vtype = GRB.BINARY, name = "x") 
model.update()

obj = sum(c[i] * x[i] for i in range(c.shape[0]))
# 或"MINIMIZE"
model.setObjective(obj, GRB.MAXIMIZE)
# constraint
model.addConstr( quicksum(A[i] * x[i] for i in range(A.shape[0])) <= b) 

<gurobi.Constr *Awaiting Model Update*>

### 輸出結果

變數<br>
t: 開始時間<br>
t_opt: 結束時間<br>
out: 計算結果<br>
total: 總價值<br>

In [133]:

# 用作開始時間的當前時間
t = time.process_time()
model.optimize()
t_opt = time.process_time()
# 結束時間減去開始時間
print(f"Optimization time: {t_opt - t} seconds")
# 將優化問題存儲為文本文件。可以檢查問題
model.write('output/Knapsack.lp')

# 將變量名稱和值附加到輸出列表
out = [] 
total = 0

for i in model.getVars():
    # 將變量名稱和值附加到輸出列表
    out.append([i.varName, i.x]) 
print(out)
# 加總
for i in range(sample):
    total += c[i]*out[i][1]
    print(int(total))
# 存儲為 Pandas 數據框
out.append(["total_value",total])
df = pd.DataFrame(out) 

# 輸出為 Excel（或 csv）文件
df.to_csv('output/Knapsack.csv') 
t_rec = time.process_time()
print("Recording output time:", t_rec - t_opt, " seconds")

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 15 columns and 15 nonzeros
Model fingerprint: 0xea7723e7
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [7e+01, 1e+02]
  Objective range  [1e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+02, 8e+02]


Presolve removed 1 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 4 available processors)

Solution count 2: 1458 1249 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.458000000000e+03, best bound 1.458000000000e+03, gap 0.0000%
Optimization time: 0.03258576099999999 seconds
[['x[0]', 1.0], ['x[1]', 0.0], ['x[2]', 1.0], ['x[3]', 0.0], ['x[